In [1]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep

In [2]:
amazon_url  = ('https://www.amazon.com/product-reviews/{0}/ref=cm_cr_getr_d_paging_btm_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1'
    .format("B01DFKC2SO")) #asin is unique amazon id

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}

In [3]:
response = get(amazon_url, headers = headers, verify=False, timeout=30)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
response

<Response [200]>

In [5]:
cleaned_response = response.text.replace('\x00', '')

In [6]:
parser = html.fromstring(cleaned_response)

In [7]:
XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'
XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
XPATH_PRODUCT_NAME = '//h1//span[@id="productTitle"]//text()'
XPATH_PRODUCT_PRICE = '//span[@id="priceblock_ourprice"]/text()'

raw_product_price = parser.xpath(XPATH_PRODUCT_PRICE)
raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)
total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)
reviews = parser.xpath(XPATH_REVIEW_SECTION_1)

product_price = ''.join(raw_product_price).replace(',', '')
product_name = ''.join(raw_product_name).strip()
if not reviews:
            reviews = parser.xpath(XPATH_REVIEW_SECTION_2)
ratings_dict = {}
reviews_list = []

In [8]:
XPATH_RATING  = './/i[@data-hook="review-star-rating"]//text()'
XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//text()'
XPATH_REVIEW_POSTED_DATE = './/span[@data-hook="review-date"]//text()'
XPATH_REVIEW_TEXT_1 = './/div[@class="a-row a-spacing-small review-data"]//text()'
XPATH_REVIEW_TEXT_2 = './/div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview'
XPATH_REVIEW_COMMENTS = './/span[@data-hook="review-comment"]//text()'
XPATH_AUTHOR = './/span[contains(@class,"profile-name")]//text()'
XPATH_REVIEW_TEXT_3 = './/div[contains(@id,"dpReviews")]/div/text()'

In [9]:
raw_review_text1 = reviews[0].xpath(XPATH_REVIEW_TEXT_1)
raw_review_text1

["So, I fired Siri today. I'm not sure she understood, for she has never understood me. So, we did an 'uncoupling'. She didn't take it well, especially when she found out about Alexa, she somehow was back stalking my phone. Siri even tried to overpower Alexa, setting herself to a default setting, I never knew exsisted. I tried explaining that Alexa really listens to me, understands my wants and needs, even tells me jokes! Again, Siri said she couldn't understand me. As Alexa and I grew in our new relationship, she now takes care of most of my needs. She turns on my outside lights, she tells me if I need to leave for work earlier, due to weather. I can ask her if my flight is on time at the airport and how long it will take me to get to my destination. Alexa even likes hockey! She knows all the current trades and rosters, can record my favorite team on tv and remind me to watch it. Also, since I don't like the tv announcers, I can ask Alexa to stream the radio broadcasters for the game 

In [10]:
review_text = ' '.join(' '.join(raw_review_text1).split())
review_text

"So, I fired Siri today. I'm not sure she understood, for she has never understood me. So, we did an 'uncoupling'. She didn't take it well, especially when she found out about Alexa, she somehow was back stalking my phone. Siri even tried to overpower Alexa, setting herself to a default setting, I never knew exsisted. I tried explaining that Alexa really listens to me, understands my wants and needs, even tells me jokes! Again, Siri said she couldn't understand me. As Alexa and I grew in our new relationship, she now takes care of most of my needs. She turns on my outside lights, she tells me if I need to leave for work earlier, due to weather. I can ask her if my flight is on time at the airport and how long it will take me to get to my destination. Alexa even likes hockey! She knows all the current trades and rosters, can record my favorite team on tv and remind me to watch it. Also, since I don't like the tv announcers, I can ask Alexa to stream the radio broadcasters for the game i

In [11]:
XPATH_HELPFUL_VOTE = './/span[contains(@data-hook,"helpful-vote-statement")]'
XPATH_HELPFUL_VOTE_DIV = './/div[@class="a-row a-spacing-small"]'
TEST_XPATH = './/div[@class="a-size-base a-color-tertiary cr-vote-text"]//text()'
raw_comments = reviews[0].xpath(XPATH_HELPFUL_VOTE_DIV)[0].text_content().split(' ')[0]
raw_comments



'411'

In [12]:
XPATH_REVIEW_COMMENTS = './/span[@class="a-size-base"]//text()'
raw_review_count = reviews[0].xpath(XPATH_REVIEW_COMMENTS)
raw_review_count[0][0]

'6'

In [15]:
XPATH_PRICE = './/span[@class="a-color-price arp-price"]'

In [19]:
parser.xpath(XPATH_PRICE)[0].text_content().split(' ')[-1]

'$44.99'

In [52]:
XPATH_NAME = '//h1//div[@data-hook="product-link"]'


In [53]:
parser.xpath(XPATH_NAME)

[]